In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import tensorflow.compat.v1 as tf
from tensorflow import keras
import time
from sklearn.metrics import brier_score_loss

In [2]:
df = pd.read_csv('final_data.csv')
df.head()

,Unnamed: 0,HeartDiseaseorAttack,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,MentHlth,PhysHlth,DiffWalk,NoDiabetes,PreDiabetes,HasDiabetes,ExcellentGenHealth,VeryGoodGenHealth,GoodGenHealth,FairGenHealth,PoorGenHealth,Age18to24,Age25to29,Age30to34,Age35to39,Age40to44,Age45to49,Age50to54,Age55to59,Age60to64,Age65to69,Age70to74,Age75to79,Age80Plus,EducationNoElementary,EducationElementary,EducationSomeHS,EducationHSGrad,EducationSomeCollege,EducationCollegeGrad,IncomeUnder10k,Income10kto15k,Income15kto20k,Income20kto25k,Income25kto35k,Income35kto50k,Income50kto75k,IncomeOver75k,Male,Female
0,0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,18.0,15.0,1.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0
1,1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0
2,2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,30.0,30.0,1.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0
3,3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
4,4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0


In [3]:
def stratShuffleSplit(X, y):
    s = StratifiedShuffleSplit(test_size=.2, random_state=42)
    for train_index, test_index in s.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    return X_train, X_test, y_train, y_test

X_raw, y_raw = df.drop(['Unnamed: 0','HeartDiseaseorAttack'],axis=1), df['HeartDiseaseorAttack']
X_raw = X_raw.to_numpy()
y_raw = y_raw.to_numpy()

X_train, X_test, y_train, y_test = stratShuffleSplit(X_raw, y_raw)

print(X_train.shape)
print(X_test.shape)
y_train = y_train.reshape(-1,1)
print(y_train.shape)
y_test = y_test.reshape(-1,1)
print(y_test.shape)

#X_train, X_validation, X_test, y_train, y_validation, y_test = X_train[:20000], X_test[:5000], X_test[5000:10000], y_train[:20000], y_test[:5000], y_test[5000:10000]
X_train, X_validation, X_test, y_train, y_validation, y_test = X_train, X_test[:25000], X_test[:25000], y_train, y_test[:25000], y_test[:25000]

(202944, 52)
(50736, 52)
(202944, 1)
(50736, 1)


In [4]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_random_seed(42)

def firstModel(nHidden, nNeurons, inputShape):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=inputShape))
    model.add(keras.layers.Dense(nNeurons, activation='elu', kernel_initializer='he_normal'))
    for layer in range(nHidden - 1):
        model.add(keras.layers.Dense(nNeurons, activation='elu', kernel_initializer='he_normal'))
    model.add(keras.layers.Dense(2, activation='softmax'))
    optimizer = keras.optimizers.SGD(learning_rate=1e-3)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)
keras_clf = firstModel(2,52,[52])

t = time.time()
keras_clf.fit(X_train, y_train, epochs=10, validation_data=(X_validation, y_validation), callbacks=[early_stopping_cb])
e = time.time() - t
print(str(int(e))+'s')

brier_score_loss(y_test,keras_clf.predict(X_test)[:,1])

Epoch 1/10
6342/6342 [==============================] - 48s 7ms/step - loss: 0.2805 - accuracy: 0.9039 - val_loss: 0.2636 - val_accuracy: 0.9044
Epoch 2/10
6342/6342 [==============================] - 37s 6ms/step - loss: 0.2586 - accuracy: 0.9052 - val_loss: 0.2604 - val_accuracy: 0.9047
Epoch 3/10
6342/6342 [==============================] - 36s 6ms/step - loss: 0.2543 - accuracy: 0.9055 - val_loss: 0.2512 - val_accuracy: 0.9045
Epoch 4/10
6342/6342 [==============================] - 37s 6ms/step - loss: 0.2513 - accuracy: 0.9058 - val_loss: 0.2501 - val_accuracy: 0.9046
Epoch 5/10
6342/6342 [==============================] - 37s 6ms/step - loss: 0.2495 - accuracy: 0.9061 - val_loss: 0.2549 - val_accuracy: 0.8998
Epoch 6/10
6342/6342 [==============================] - 38s 6ms/step - loss: 0.2480 - accuracy: 0.9063 - val_loss: 0.2718 - val_accuracy: 0.9045
Epoch 7/10
6342/6342 [==============================] - 36s 6ms/step - loss: 0.2470 - accuracy: 0.9061 - val_loss: 0.2695 - val_ac

0.07345507218855067

In [5]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_random_seed(42)

def secondModel(nHidden, nNeurons, inputShape):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=inputShape))
    model.add(keras.layers.Dense(nNeurons, activation='elu', kernel_initializer='he_normal'))
    for layer in range(nHidden - 1):
        model.add(keras.layers.Dense(nNeurons, activation='elu', kernel_initializer='he_normal'))
    model.add(keras.layers.Dense(2, activation='softmax'))
    optimizer = keras.optimizers.Nadam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999)
    #model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['mean_squared_error'])
    return model

early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)
keras_clf = secondModel(2,52,[52])

t = time.time()
keras_clf.fit(X_train, y_train, epochs=10, validation_data=(X_validation, y_validation), callbacks=[early_stopping_cb])
e = time.time() - t
print(str(int(e))+'s')

brier_score_loss(y_test,keras_clf.predict(X_test)[:,1])

Epoch 1/10
6342/6342 [==============================] - 70s 11ms/step - loss: 0.2493 - mean_squared_error: 0.4314 - val_loss: 0.2443 - val_mean_squared_error: 0.4224
Epoch 2/10
6342/6342 [==============================] - 75s 12ms/step - loss: 0.2412 - mean_squared_error: 0.4300 - val_loss: 0.2440 - val_mean_squared_error: 0.4382
Epoch 3/10
6342/6342 [==============================] - 111s 18ms/step - loss: 0.2398 - mean_squared_error: 0.4298 - val_loss: 0.2425 - val_mean_squared_error: 0.4370
Epoch 4/10
6342/6342 [==============================] - 89s 14ms/step - loss: 0.2392 - mean_squared_error: 0.4299 - val_loss: 0.2399 - val_mean_squared_error: 0.4293
Epoch 5/10
6342/6342 [==============================] - 78s 12ms/step - loss: 0.2386 - mean_squared_error: 0.4300 - val_loss: 0.2405 - val_mean_squared_error: 0.4266
Epoch 6/10
6342/6342 [==============================] - 74s 12ms/step - loss: 0.2383 - mean_squared_error: 0.4301 - val_loss: 0.2420 - val_mean_squared_error: 0.4376
Epo

0.07109413723784622

In [6]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_random_seed(42)

def thirdModel(nHidden, nNeurons, inputShape):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=inputShape))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(nNeurons, activation='elu', kernel_initializer='he_normal', use_bias=False))
    for layer in range(nHidden - 1):
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dense(nNeurons, activation='elu', kernel_initializer='he_normal', use_bias=False))
        model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(2, activation='softmax'))
    optimizer = keras.optimizers.SGD(learning_rate=1e-3)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)
keras_clf = thirdModel(2,52,[52])

t = time.time()
keras_clf.fit(X_train, y_train, epochs=10, validation_data=(X_validation, y_validation), callbacks=[early_stopping_cb])
e = time.time() - t
print(str(int(e))+'s')

brier_score_loss(y_test,keras_clf.predict(X_test)[:,1])

Epoch 1/10
6342/6342 [==============================] - 63s 10ms/step - loss: 0.3327 - accuracy: 0.8724 - val_loss: 0.2650 - val_accuracy: 0.9015
Epoch 2/10
6342/6342 [==============================] - 61s 10ms/step - loss: 0.2597 - accuracy: 0.9039 - val_loss: 0.2563 - val_accuracy: 0.9021
Epoch 3/10
6342/6342 [==============================] - 59s 9ms/step - loss: 0.2552 - accuracy: 0.9046 - val_loss: 0.2530 - val_accuracy: 0.9028
Epoch 4/10
6342/6342 [==============================] - 62s 10ms/step - loss: 0.2527 - accuracy: 0.9048 - val_loss: 0.2509 - val_accuracy: 0.9034
Epoch 5/10
6342/6342 [==============================] - 59s 9ms/step - loss: 0.2508 - accuracy: 0.9050 - val_loss: 0.2492 - val_accuracy: 0.9040
Epoch 6/10
6342/6342 [==============================] - 60s 9ms/step - loss: 0.2493 - accuracy: 0.9054 - val_loss: 0.2483 - val_accuracy: 0.9043
Epoch 7/10
6342/6342 [==============================] - 57s 9ms/step - loss: 0.2487 - accuracy: 0.9055 - val_loss: 0.2473 - val

0.07255016922714443

In [7]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_random_seed(42)

def fourthModel(nHidden, nNeurons, inputShape):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=inputShape))
    model.add(keras.layers.Dense(nNeurons, activation='selu', kernel_initializer='lecun_normal'))
    for layer in range(nHidden - 1):
        model.add(keras.layers.Dense(nNeurons, activation='selu', kernel_initializer='lecun_normal'))
    model.add(keras.layers.Dense(2, activation='softmax'))
    optimizer = keras.optimizers.SGD(learning_rate=1e-3)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)
keras_clf = fourthModel(2,52,[52])

t = time.time()
keras_clf.fit(X_train, y_train, epochs=10, validation_data=(X_validation, y_validation), callbacks=[early_stopping_cb])
e = time.time() - t
print(str(int(e))+'s')

brier_score_loss(y_test,keras_clf.predict(X_test)[:,1])

Epoch 1/10
6342/6342 [==============================] - 36s 6ms/step - loss: 0.2794 - accuracy: 0.9045 - val_loss: 0.2656 - val_accuracy: 0.9043
Epoch 2/10
6342/6342 [==============================] - 36s 6ms/step - loss: 0.2575 - accuracy: 0.9057 - val_loss: 0.2582 - val_accuracy: 0.9047
Epoch 3/10
6342/6342 [==============================] - 35s 6ms/step - loss: 0.2519 - accuracy: 0.9063 - val_loss: 0.2498 - val_accuracy: 0.9040
Epoch 4/10
6342/6342 [==============================] - 35s 5ms/step - loss: 0.2490 - accuracy: 0.9061 - val_loss: 0.2485 - val_accuracy: 0.9049
Epoch 5/10
6342/6342 [==============================] - 35s 5ms/step - loss: 0.2474 - accuracy: 0.9066 - val_loss: 0.2518 - val_accuracy: 0.9014
Epoch 6/10
6342/6342 [==============================] - 37s 6ms/step - loss: 0.2461 - accuracy: 0.9066 - val_loss: 0.2673 - val_accuracy: 0.9042
Epoch 7/10
6342/6342 [==============================] - 34s 5ms/step - loss: 0.2453 - accuracy: 0.9066 - val_loss: 0.2664 - val_ac

0.07302706907501723

In [8]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_random_seed(42)

def fifthModel(nHidden, nNeurons, inputShape):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=inputShape))
    model.add(keras.layers.AlphaDropout(rate=0.2))
    model.add(keras.layers.Dense(nNeurons, activation='selu', kernel_initializer='lecun_normal'))
    for layer in range(nHidden - 1):
        model.add(keras.layers.AlphaDropout(rate=0.2))
        model.add(keras.layers.Dense(nNeurons, activation='selu', kernel_initializer='lecun_normal'))
        model.add(keras.layers.AlphaDropout(rate=0.2))
    model.add(keras.layers.Dense(2, activation='softmax'))
    optimizer = keras.optimizers.SGD(learning_rate=1e-3)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)
keras_clf = fifthModel(2,52,[52])

t = time.time()
keras_clf.fit(X_train, y_train, epochs=10, validation_data=(X_validation, y_validation), callbacks=[early_stopping_cb])
e = time.time() - t
print(str(int(e))+'s')

brier_score_loss(y_test,keras_clf.predict(X_test)[:,1])

Epoch 1/10
6342/6342 [==============================] - 39s 6ms/step - loss: 0.3888 - accuracy: 0.8678 - val_loss: 0.3216 - val_accuracy: 0.8996
Epoch 2/10
6342/6342 [==============================] - 39s 6ms/step - loss: 0.3246 - accuracy: 0.8983 - val_loss: 0.3095 - val_accuracy: 0.9025
Epoch 3/10
6342/6342 [==============================] - 38s 6ms/step - loss: 0.3149 - accuracy: 0.9026 - val_loss: 0.3089 - val_accuracy: 0.9034
Epoch 4/10
6342/6342 [==============================] - 38s 6ms/step - loss: 0.3114 - accuracy: 0.9041 - val_loss: 0.3058 - val_accuracy: 0.9034
Epoch 5/10
6342/6342 [==============================] - 38s 6ms/step - loss: 0.3088 - accuracy: 0.9048 - val_loss: 0.3060 - val_accuracy: 0.9041
Epoch 6/10
6342/6342 [==============================] - 38s 6ms/step - loss: 0.3076 - accuracy: 0.9052 - val_loss: 0.3041 - val_accuracy: 0.9040
Epoch 7/10
6342/6342 [==============================] - 39s 6ms/step - loss: 0.3057 - accuracy: 0.9054 - val_loss: 0.3038 - val_ac

0.0840730052710205

In [9]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_random_seed(42)

def sixthModel(nHidden, nNeurons, inputShape):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=inputShape))
    model.add(keras.layers.AlphaDropout(rate=0.2))
    model.add(keras.layers.Dense(nNeurons, activation='selu', kernel_initializer='lecun_normal'))
    for layer in range(nHidden - 1):
        model.add(keras.layers.AlphaDropout(rate=0.2))
        model.add(keras.layers.Dense(nNeurons, activation='selu', kernel_initializer='lecun_normal'))
        model.add(keras.layers.AlphaDropout(rate=0.2))
    model.add(keras.layers.Dense(2, activation='softmax'))
    optimizer = keras.optimizers.SGD(learning_rate=1e-3)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)
keras_clf = sixthModel(2,52,[52])

t = time.time()
keras_clf.fit(X_train, y_train, epochs=10, validation_data=(X_validation, y_validation), callbacks=[early_stopping_cb])
e = time.time() - t
print(str(int(e))+'s')

brier_score_loss(y_test,keras_clf.predict(X_test)[:,1])

Epoch 1/10
6342/6342 [==============================] - 67s 10ms/step - loss: 0.3888 - accuracy: 0.8678 - val_loss: 0.3216 - val_accuracy: 0.8997
Epoch 2/10
6342/6342 [==============================] - 66s 10ms/step - loss: 0.3246 - accuracy: 0.8983 - val_loss: 0.3095 - val_accuracy: 0.9026
Epoch 3/10
6342/6342 [==============================] - 67s 11ms/step - loss: 0.3149 - accuracy: 0.9026 - val_loss: 0.3089 - val_accuracy: 0.9034
Epoch 4/10
6342/6342 [==============================] - 67s 11ms/step - loss: 0.3114 - accuracy: 0.9041 - val_loss: 0.3058 - val_accuracy: 0.9034
Epoch 5/10
6342/6342 [==============================] - 67s 11ms/step - loss: 0.3088 - accuracy: 0.9048 - val_loss: 0.3060 - val_accuracy: 0.9041
Epoch 6/10
6342/6342 [==============================] - 67s 11ms/step - loss: 0.3076 - accuracy: 0.9052 - val_loss: 0.3041 - val_accuracy: 0.9040
Epoch 7/10
6342/6342 [==============================] - 67s 11ms/step - loss: 0.3057 - accuracy: 0.9054 - val_loss: 0.3038 -

0.08407321862161195